In [1]:
import os  
import pdfplumber
import numpy as np
import pandas as pd
from WindPy import w


In [2]:

w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒 

w.isconnected() # 判断WindPy是否已经登录成功


Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [3]:
#改金额单位
def mc(amount):
    value = ''
    amount = float(amount)
    if abs(amount) >= 10000 and abs(amount) < 100000000:
        val = round(amount/10000)
        value = str(val) + "万元"
    elif abs(amount) >= 100000000:
        val = round(amount/100000000,2)
        value = str(val) + "亿元"
        
    return value 

In [4]:
def file_name(file_dir):  
    L=[]  
    for root, dirs, files in os.walk(file_dir): 
        for file in files: 
            if os.path.splitext(file)[1] == '.pdf': 
                L.append(os.path.join(root, file)) 
    return L


In [5]:
filename = input("文件路径：")

if filename.find('.S')!= -1 :
    a = filename.find('.S') + 3
    b = filename[:a]
    c = b[(a-9):]
    code = c



code
        




文件路径：2020-08-01-300378.SZ-鼎捷软件_2020年半年度报告.pdf


'300378.SZ'

In [6]:
pdf = pdfplumber.open(filename)

In [7]:
############################ 主要会计数据和财务指标 ###############################

# 自动查找页码

for i in range(2,10):
    p0 = pdf.pages[i]
    pp = p0.extract_text(x_tolerance=3, y_tolerance=3)
    if pp.find("主要会计数据")> -1:
        page_num = i
        break
        
        

In [8]:


p0 = pdf.pages[page_num]
table = p0.extract_table()


In [9]:
table

[['', '本报告期', '上年同期', '本报告期比上年同期增\n减'],
 ['营业收入（元）', '602,033,403.01', '656,878,885.99', '-8.35%'],
 ['归属于上市公司股东的净利润（元）', '-14,743,221.54', '40,217,245.88', '-136.66%'],
 ['归属于上市公司股东的扣除非经常性\n损益后的净利润（元）',
  '-23,292,776.81',
  '31,889,938.97',
  '-173.04%'],
 ['经营活动产生的现金流量净额（元）', '-87,590,201.79', '-29,733,596.27', ''],
 ['基本每股收益（元/股）', '-0.06', '0.15', '-140.00%'],
 ['稀释每股收益（元/股）', '-0.06', '0.15', '-140.00%'],
 ['加权平均净资产收益率', '-1.06%', '3.06%', '-4.12%'],
 ['', '本报告期末', '上年度末', '本报告期末比上年度末\n增减'],
 ['总资产（元）', '2,150,589,822.09', '2,269,797,270.27', '-5.25%'],
 ['归属于上市公司股东的净资产（元）', '1,376,239,313.88', '1,401,929,568.89', '-1.83%']]

In [10]:

df = pd.DataFrame(table[1:6], columns=table[0])
# for column in ["Effective", "Received"]:
#     df[column] = df[column].str.replace(" ", "")
df



,,本报告期,上年同期,本报告期比上年同期增\n减
0,营业收入（元）,"602,033,403.01","656,878,885.99",-8.35%
1,归属于上市公司股东的净利润（元）,"-14,743,221.54","40,217,245.88",-136.66%
2,归属于上市公司股东的扣除非经常性\n损益后的净利润（元）,"-23,292,776.81","31,889,938.97",-173.04%
3,经营活动产生的现金流量净额（元）,"-87,590,201.79","-29,733,596.27",
4,基本每股收益（元/股）,-0.06,0.15,-140.00%


In [11]:
# 洗数据。。。
data = df.values

#设置变量index，保证扣非的读取
test_var1 = data[3,0]
test_var2 = data[2,0]
index = []
if test_var1!= None and test_var2.find("经常性")!= -1:
    index = int(2)
elif test_var1==None and test_var2.find("经常性")!= -1:
    index = int(3)
    
    
percent = {}

#去除逗号
for i in [0,1,index]:
    for j in [1,2,3]:
        data[i,j] = data[i,j].replace(',','')
        percent[i] = data[i,3]
        if percent[i].find('%')!=-1:
            percent[i] = percent[i].replace('%','')
        



In [12]:
# 修改数据类型
percent[0] = float(percent[0])
percent[1] = float(percent[1])
percent[index] = float(percent[index])

# 判断增减
updown = {}
for i in [0,1,index]:
    if percent[i]>=0:
        updown[i]="增加"
    else:
        updown[i]="减少"
        percent[i]=abs(percent[i])



In [13]:
################################ 营业利润 #########################################
# 从合并利润表里提取
page_num1 = 0
for z in range(20,80):
    p1 = pdf.pages[z]
    pp1 = p1.extract_text(x_tolerance=3, y_tolerance=3)
    if pp1.find("三、营业利润（亏损")> -1:
        page_num1 = z
        break
        

# 提取表格内容
p1 = pdf.pages[page_num1]
table1 = p1.extract_table()
table1


[['“－”号填列）', '', ''],
 ['公允价值变动收益（损失以\n“－”号填列）', '', '2,387,815.07'],
 ['信用减值损失（损失以“-”\n号填列）', '-13,085,452.64', '-18,653,617.75'],
 ['资产减值损失（损失以“-”\n号填列）', '', ''],
 ['资产处置收益（损失以“-”\n号填列）', '', '3,480.00'],
 ['三、营业利润（亏损以“－”号填列）', '-4,190,454.71', '56,865,424.59'],
 ['加：营业外收入', '1,265,522.00', '1,007,673.82'],
 ['减：营业外支出', '528,448.33', '751,183.39'],
 ['四、利润总额（亏损总额以“－”号\n填列）', '-3,453,381.04', '57,121,915.02'],
 ['减：所得税费用', '11,977,424.96', '16,292,799.40'],
 ['五、净利润（净亏损以“－”号填列）', '-15,430,806.00', '40,829,115.62'],
 ['（一）按经营持续性分类', '', ''],
 ['1.持续经营净利润（净亏损以\n“－”号填列）', '-15,430,806.00', '40,884,643.60'],
 ['2.终止经营净利润（净亏损以\n“－”号填列）', '', '-55,527.98'],
 ['（二）按所有权归属分类', '', ''],
 ['1.归属于母公司所有者的净利润', '-14,743,221.54', '40,217,245.88'],
 ['2.少数股东损益', '-687,584.46', '611,869.74'],
 ['六、其他综合收益的税后净额', '13,177,003.84', '-15,955,513.08'],
 ['归属母公司所有者的其他综合收\n益的税后净额', '13,111,039.06', '-15,179,401.23'],
 ['（一）不能重分类进损益的其他\n综合收益', '', ''],
 ['1.重新计量设定受益计划\n变动额', '', ''],
 ['2.权益法下不能转损益的\n其他综合收益',

In [14]:
# 把表格装进dataframe
df1 = pd.DataFrame(table1, columns=table1[0])

df1


,“－”号填列）,,
0,“－”号填列）,,
1,公允价值变动收益（损失以\n“－”号填列）,,"2,387,815.07"
2,信用减值损失（损失以“-”\n号填列）,"-13,085,452.64","-18,653,617.75"
3,资产减值损失（损失以“-”\n号填列）,,
4,资产处置收益（损失以“-”\n号填列）,,"3,480.00"
5,三、营业利润（亏损以“－”号填列）,"-4,190,454.71","56,865,424.59"
6,加：营业外收入,"1,265,522.00","1,007,673.82"
7,减：营业外支出,"528,448.33","751,183.39"
8,四、利润总额（亏损总额以“－”号\n填列）,"-3,453,381.04","57,121,915.02"
9,减：所得税费用,"11,977,424.96","16,292,799.40"


In [15]:
# 还是洗数据

#查找营业利润在第几行：

dfsize = df1.count()[0]

for i in range(0,dfsize-1):
    if df1.iloc[i][0] == "三、营业利润（亏损以“－”号填列）":
        loc = i
        break

# 读取该行数据
# 分离附注

profit_this_temp = df1.iloc[loc][1]
profit_last_temp = df1.iloc[loc][2]

# 去逗号
profit_this = float(profit_this_temp.replace(',',''))
profit_last = float(profit_last_temp.replace(',',''))

# 计算增减比率
profit_var = round(100*(profit_this - profit_last)/profit_last,2)


# 判断增减

if profit_var > 0:
    updown_p = '增加'
elif profit_var < 0:
    updown_p = '减少'

#特殊情况
if profit_this < 0 and profit_this < 0:
    updown_p = '减少'
    
    

In [16]:
Revenue = {'obj':data[0,0],'this':data[0,1],'last':data[0,2],'varpct':percent[0]} #把数据写入dict

NetProfit_sh = {'obj':data[1,0],'this':data[1,1],'last':data[1,2],'varpct':percent[1]} #把数据写入dict

NetProfit_sh_loss = {'obj':data[2,0],'this':data[index,1],'last':data[index,2],'varpct':percent[index]} #把数据写入dict

Profit = {'obj':df1.iloc[loc][0],'this':profit_this,'last':profit_last,'varpct':profit_var}




In [17]:
# 上半年生成
line1_h1 = "20H1营业总收入" + mc(Revenue['this']) + ",上年同期为" + mc(Revenue['last']) + ",同比" + updown[0] + str(percent[0]) + '%'
line2_h1 = "20H1营业利润" + mc(Profit['this']) + ",上年同期为" + mc(Profit['last']) + ",同比" + updown_p + str(Profit['varpct']) + '%'
line3_h1 = "20H1归母净利润" + mc(NetProfit_sh['this']) + ",上年同期为" + mc(NetProfit_sh['last']) + ",同比" + updown[1] + str(percent[1]) + '%'
line4_h1 = "20H1扣非归母净利润" + mc(NetProfit_sh_loss['this']) + ",上年同期为" + mc(NetProfit_sh_loss['last']) + ",同比" + updown[index] + str(percent[index]) + '%'

In [18]:
######### 以下是wind部分 ##########

In [19]:
#营业总收入 _y
# 从wind查找q1数据

wsddata1 = w.wsd(code, "tot_oper_rev", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata1


.ErrorCode=0
.Codes=[300378.SZ]
.Fields=[TOT_OPER_REV]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[255193701.24,656878885.99,1008139170.61,1466718146.65,231365371.72]]

In [20]:
#计算q2数据
lastq1_y = wsddata1.Data[0][0] #选取2019年q1数据
lasth1_y = wsddata1.Data[0][1] #选取2019年h1数据
thisq1_y = wsddata1.Data[0][4] #选取2020年q1数据
thish1_y = float(Revenue['this'])
lastq2_y = lasth1_y - lastq1_y    #计算2019年q2数据
thisq2_y = thish1_y - thisq1_y 

#计算同比增长/减少
growth_year = {}
growth_year[0] = (thisq2_y - lastq2_y)/abs(lastq2_y) * 100

In [21]:
#归母净利润 _g
# 从wind查找q1数据
wsddata2 = w.wsd(code, "np_belongto_parcomsh", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata2



.ErrorCode=0
.Codes=[300378.SZ]
.Fields=[NP_BELONGTO_PARCOMSH]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[-12829850.73,40217245.88,61638468.41,103363164.79,-52788990.04]]

In [22]:

#计算q2数据
lastq1_g = wsddata2.Data[0][0] #选取2019年q1数据
lasth1_g = wsddata2.Data[0][1] #选取2019年h1数据
thisq1_g = wsddata2.Data[0][4] #选取2020年q1数据
thish1_g = float(NetProfit_sh['this'])
lastq2_g = lasth1_g - lastq1_g     #计算2019年q2数据
thisq2_g = thish1_g - thisq1_g 

#计算同比增长/减少
growth_year[1] = (thisq2_g - lastq2_g)/abs(lastq2_g) * 100

In [23]:
#扣非归母净利润 _k
# 从wind查找q1数据
wsddata3 = w.wsd(code, "deductedprofit", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata3


.ErrorCode=0
.Codes=[300378.SZ]
.Fields=[DEDUCTEDPROFIT]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[-14511265.68,31889938.97,53311161.5,89076813.26,-58781953.29]]

In [24]:
#计算q2数据
lastq1_k = wsddata3.Data[0][0] #选取2019年q1数据
lasth1_k = wsddata3.Data[0][1] #选取2019年h1数据
thisq1_k = wsddata3.Data[0][4] #选取2020年q1数据
thish1_k = float(NetProfit_sh_loss['this'])
lastq2_k = lasth1_k - lastq1_k     #计算2019年q2数据
thisq2_k = thish1_k - thisq1_k 

#计算同比增长/减少
growth_year[2] = (thisq2_k - lastq2_k)/abs(lastq2_k) * 100

In [25]:
#营业利润 _lr
# 从wind查找q1数据
wsddata4 = w.wsd(code, "opprofit", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata4


.ErrorCode=0
.Codes=[300378.SZ]
.Fields=[OPPROFIT]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[-11494525.26,56865424.59,84271276.34,142631511.62,-52613519.03]]

In [26]:
#计算q2数据
lastq1_lr = wsddata4.Data[0][0] #选取2019年q1数据
lasth1_lr = wsddata4.Data[0][1] #选取2019年h1数据
thisq1_lr = wsddata4.Data[0][4] #选取2020年q1数据
thish1_lr = float(Profit['this'])
lastq2_lr = lasth1_lr - lastq1_lr     #计算2019年q2数据
thisq2_lr = thish1_lr - thisq1_lr 

#计算同比增长/减少
growth_year_p = (thisq2_lr - lastq2_lr)/abs(lastq2_lr) * 100

In [27]:
# 判断增减
updown_q2 = {}
for i in [0,1,2]:
    if growth_year[i]>=0:
        updown_q2[i]="增加"
    else:
        updown_q2[i]="减少"
        growth_year[i]=abs(growth_year[i])

if growth_year_p >=0:
    updown_p_q2="增加"
else:
    updown_p_q2="减少"



In [28]:
# 二季度生成
line1_q2 = "20Q2营业总收入" + mc(thisq2_y) + ",上年同期为" + mc(lastq2_y) + ",同比" + updown_q2[0] + str(round(growth_year[0],2)) + '%'
line2_q2 = "20Q2营业利润" + mc(thisq2_lr) + ",上年同期为" + mc(lastq2_lr) + ",同比" + updown_p_q2 + str(round(growth_year_p,2)) + '%'
line3_q2 = "20Q2归母净利润" + mc(thisq2_g) + ",上年同期为" + mc(lastq2_g) + ",同比" + updown_q2[1] + str(round(growth_year[1],2)) + '%'
line4_q2 = "20Q2扣非归母净利润" + mc(thisq2_k) + ",上年同期为" + mc(lastq2_k) + ",同比" + updown_q2[2] + str(round(growth_year[2],2)) + '%'

In [29]:
############################### 费用 ################################

# 从合并利润表里提取
page_num2 = 0
for y in range(20,80):
    p2 = pdf.pages[y]
    pp2 = p2.extract_text(x_tolerance=3, y_tolerance=3)
    if pp2.find("研发费用")> -1:
        page_num2 = y
        break
        

# 提取表格内容
p2 = pdf.pages[page_num2]
table2 = p2.extract_table()
table2


[['项目', '2020年半年度', '2019年半年度'],
 ['一、营业总收入', '602,033,403.01', '656,878,885.99'],
 ['其中：营业收入', '602,033,403.01', '656,878,885.99'],
 ['利息收入', '', ''],
 ['已赚保费', '', ''],
 ['手续费及佣金收入', '', ''],
 ['二、营业总成本', '609,525,621.86', '599,175,095.42'],
 ['其中：营业成本', '106,120,155.54', '107,440,218.22'],
 ['利息支出', '', ''],
 ['手续费及佣金支出', '', ''],
 ['退保金', '', ''],
 ['赔付支出净额', '', ''],
 ['提取保险责任准备金净\n额', '', ''],
 ['保单红利支出', '', ''],
 ['分保费用', '', ''],
 ['税金及附加', '3,326,031.69', '3,642,319.46'],
 ['销售费用', '328,365,541.50', '318,055,386.54'],
 ['管理费用', '97,031,395.85', '93,426,013.41'],
 ['研发费用', '76,122,929.18', '76,153,256.90'],
 ['财务费用', '-1,440,431.90', '457,900.89'],
 ['其中：利息费用', '2,080,187.35', '2,528,942.76'],
 ['利息收入', '2,634,320.37', '2,025,662.28'],
 ['加：其他收益', '12,938,418.26', '12,489,680.16'],
 ['投资收益（损失以“－”号\n填列）', '3,448,798.52', '2,934,276.54'],
 ['其中：对联营企业和合营企\n业的投资收益', '-680,719.29', '-117,156.33'],
 ['以摊余成本计量的金\n融资产终止确认收益', '', ''],
 ['汇兑收益（损失以“-”号填\n列）', '', ''],
 ['净敞口套期收益（损失以', '

In [30]:
# 把表格装进dataframe
df2 = pd.DataFrame(table2, columns=table2[0])

df2


,项目,2020年半年度,2019年半年度
0,项目,2020年半年度,2019年半年度
1,一、营业总收入,"602,033,403.01","656,878,885.99"
2,其中：营业收入,"602,033,403.01","656,878,885.99"
3,利息收入,,
4,已赚保费,,
5,手续费及佣金收入,,
6,二、营业总成本,"609,525,621.86","599,175,095.42"
7,其中：营业成本,"106,120,155.54","107,440,218.22"
8,利息支出,,
9,手续费及佣金支出,,


In [31]:
# 还是洗数据

#查找费用在第几行：

dfsize2 = df2.count()[0]

for i in range(0,dfsize2-1):
    if df2.iloc[i][0] == "销售费用":
        loc2 = i
        break

# 填写数据+去逗号
mktex_this = float(df2.iloc[loc2][1].replace(',',''))
adminex_this = float(df2.iloc[loc2+1][1].replace(',',''))
rndex_this = float(df2.iloc[loc2+2][1].replace(',',''))
finex_this = float(df2.iloc[loc2+3][1].replace(',',''))

mktex_last = float(df2.iloc[loc2][2].replace(',',''))
adminex_last = float(df2.iloc[loc2+1][2].replace(',',''))
rndex_last = float(df2.iloc[loc2+2][2].replace(',',''))
finex_last = float(df2.iloc[loc2+3][2].replace(',',''))

# 计算同比增减

mktex_var = round(100*(mktex_this - mktex_last)/mktex_last,2)
adminex_var = round(100*(adminex_this - adminex_last)/adminex_last,2)
rndex_var = round(100*(rndex_this - rndex_last)/rndex_last,2)
finex_var = round(100*(finex_this - finex_last)/finex_last,2)


# 判断增减
updown_ex={}
numm = 0
for i in [mktex_var,adminex_var,rndex_var,finex_var]:    
    if i > 0:
        updown_ex[numm] = '增加'
    elif i < 0:
        updown_ex[numm] = '减少'
    numm = numm+1

Revenue['this'] = 523211274.40
Revenue['last'] = 639753262.97

# 计算今年的费用率
mktex_rate_this = round(100*mktex_this/float(Revenue['this']),2)
adminex_rate_this = round(100*adminex_this/float(Revenue['this']),2)
rndex_rate_this = round(100*rndex_this/float(Revenue['this']),2)
finex_rate_this = round(100*finex_this/float(Revenue['this']),2)

# 计算去年的费用率
mktex_rate_last = round(100*mktex_last/float(Revenue['last']),2)
adminex_rate_last = round(100*adminex_last/float(Revenue['last']),2)
rndex_rate_last = round(100*rndex_last/float(Revenue['last']),2)
finex_rate_last = round(100*finex_last/float(Revenue['last']),2)

# 计算差值
mktex_rate_diff = round(mktex_rate_this - mktex_rate_last,2)
adminex_rate_diff = round(adminex_rate_this - adminex_rate_last,2)
rndex_rate_diff = round(rndex_rate_this - rndex_rate_last,2)
finex_rate_diff = round(finex_rate_this - finex_rate_last,2)

# 判断较去年同期增减
updown_rate={}
nu = 0
for i in [mktex_rate_diff,adminex_rate_diff,rndex_rate_diff,finex_rate_diff]:    
    if i > 0:
        updown_rate[nu] = '增加'
    elif i < 0:
        updown_rate[nu] = '减少'
    nu = nu+1


In [32]:
rateline1 = "销售费用" + mc(mktex_this) + "，同比" + updown_ex[0] + str(abs(mktex_var)) + '%' +"；销售费用率"+ str(mktex_rate_this)+'%'+"，较去年同期"+ updown_rate[0] + str(abs(mktex_rate_diff)) + 'pct；'
rateline2 = "管理费用" + mc(adminex_this) + "，同比" + updown_ex[1] + str(abs(adminex_var)) + '%' +"；管理费用率"+ str(adminex_rate_this)+'%'+"，较去年同期"+ updown_rate[1] + str(abs(adminex_rate_diff)) + 'pct；'
rateline3 = "研发费用" + mc(rndex_this) + "，同比" + updown_ex[2] + str(abs(rndex_var)) + '%' +"；研发费用率"+ str(rndex_rate_this)+'%'+"，较去年同期"+ updown_rate[2] + str(abs(rndex_rate_diff)) + 'pct；'
rateline4 = "财务费用" + mc(finex_this) + "，同比" + updown_ex[3] + str(abs(finex_var)) + '%' +"；财务费用率"+ str(finex_rate_this)+'%'+"，较去年同期"+ updown_rate[3] + str(abs(finex_rate_diff)) + 'pct。'

In [33]:
############################### 现金流 ################################

# 从主营业务分析表里提取
page_num3_a = 0
page_num3_b = 0
page_num3_c = 0
countera = 0
counterb = 0
counterc = 0
for w in range(20,75):
    p3 = pdf.pages[w]
    pp3 = p3.extract_text(x_tolerance=3, y_tolerance=3)
    if pp3.find("经营活动产生的现金流量净额")> -1:
        countera = countera + 1
        if countera == 1:
            page_num3_a = w
    if pp3.find("投资活动产生的现金流量净额")> -1:
        counterb = counterb + 1
        if counterb == 1:
            page_num3_b = w
        
    if pp3.find("筹资活动产生的现金流量净额")> -1:
        counterc = counterc + 1
        if counterc == 1:
            page_num3_c = w
      
        
# 寻找需要提取表格的页码
if page_num3_a == page_num3_b and page_num3_b != page_num3_c: # 如果投资活动现金流和经营现金流在同一页
    g = True
else: # 如果不在同一页（投资和筹资在一起）
    g = False
    

# 提取表格内容
p3a = pdf.pages[page_num3_a]
table3a = p3a.extract_table()
table3a

# print(page_num3_a)
# print(page_num3_b)
# print(page_num3_c)


[['向其他金融机构拆入资金净增\n加额', '', ''],
 ['收到原保险合同保费取得的现\n金', '', ''],
 ['收到再保业务现金净额', '', ''],
 ['保户储金及投资款净增加额', '', ''],
 ['收取利息、手续费及佣金的现\n金', '', ''],
 ['拆入资金净增加额', '', ''],
 ['回购业务资金净增加额', '', ''],
 ['代理买卖证券收到的现金净额', '', ''],
 ['收到的税费返还', '8,059,986.30', '8,353,306.47'],
 ['收到其他与经营活动有关的现\n金', '13,760,404.01', '7,914,710.98'],
 ['经营活动现金流入小计', '702,920,354.79', '706,305,302.97'],
 ['购买商品、接受劳务支付的现\n金', '127,371,306.50', '112,238,763.83'],
 ['客户贷款及垫款净增加额', '', ''],
 ['存放中央银行和同业款项净增\n加额', '', ''],
 ['支付原保险合同赔付款项的现\n金', '', ''],
 ['拆出资金净增加额', '', ''],
 ['支付利息、手续费及佣金的现\n金', '', ''],
 ['支付保单红利的现金', '', ''],
 ['支付给职工以及为职工支付的\n现金', '501,313,436.78', '459,613,411.39'],
 ['支付的各项税费', '61,505,241.48', '66,090,862.75'],
 ['支付其他与经营活动有关的现\n金', '100,320,571.82', '98,095,861.27'],
 ['经营活动现金流出小计', '790,510,556.58', '736,038,899.24'],
 ['经营活动产生的现金流量净额', '-87,590,201.79', '-29,733,596.27'],
 ['二、投资活动产生的现金流量：', '', ''],
 ['收回投资收到的现金', '', ''],
 ['取得投资收益收到的现金', '', '']]

In [68]:
p3c = pdf.pages[page_num3_c]
table3c = p3c.extract_tables() #多个表格
table3c[0]



[['处置固定资产、无形资产和其\n他长期资产收回的现金净额', '', '5,600.00'],
 ['处置子公司及其他营业单位收\n到的现金净额', '', ''],
 ['收到其他与投资活动有关的现\n金', '730,129,517.81', '473,051,432.87'],
 ['投资活动现金流入小计', '730,129,517.81', '473,057,032.87'],
 ['购建固定资产、无形资产和其\n他长期资产支付的现金', '3,957,030.27', '18,371,946.96'],
 ['投资支付的现金', '', ''],
 ['质押贷款净增加额', '', ''],
 ['取得子公司及其他营业单位支\n付的现金净额', '', '7,946,963.55'],
 ['支付其他与投资活动有关的现\n金', '671,000,000.00', '518,000,000.00'],
 ['投资活动现金流出小计', '674,957,030.27', '544,318,910.51'],
 ['投资活动产生的现金流量净额', '55,172,487.54', '-71,261,877.64'],
 ['三、筹资活动产生的现金流量：', '', ''],
 ['吸收投资收到的现金', '10,136,018.50', '3,681,197.80'],
 ['其中：子公司吸收少数股东投\n资收到的现金', '', ''],
 ['取得借款收到的现金', '906,541,808.85', '1,329,942,841.77'],
 ['收到其他与筹资活动有关的现\n金', '11,585,000.00', ''],
 ['筹资活动现金流入小计', '928,262,827.35', '1,333,624,039.57'],
 ['偿还债务支付的现金', '932,122,515.81', '1,582,547,885.28'],
 ['分配股利、利润或偿付利息支\n付的现金', '19,529,966.84', '17,852,626.56'],
 ['其中：子公司支付给少数股东\n的股利、利润', '', ''],
 ['支付其他与筹资活动有关的现\n金', '8,090,700.00', '64,461,677.19'],
 ['筹

In [69]:
# 把表格装进dataframe
df3a = pd.DataFrame(table3a, columns=table3a[0])
if np.size(table3c) < 3 or df3c.shape[0]*df3c.shape[1] != np.size(table3c):
    df3c = pd.DataFrame(table3c[0], columns=table3c[0][0])
else: 
    df3c = pd.DataFrame(table3c, columns=table3c[0])
    



df3c

,处置固定资产、无形资产和其\n他长期资产收回的现金净额,,"5,600.00"
0,处置固定资产、无形资产和其\n他长期资产收回的现金净额,,"5,600.00"
1,处置子公司及其他营业单位收\n到的现金净额,,
2,收到其他与投资活动有关的现\n金,"730,129,517.81","473,051,432.87"
3,投资活动现金流入小计,"730,129,517.81","473,057,032.87"
4,购建固定资产、无形资产和其\n他长期资产支付的现金,"3,957,030.27","18,371,946.96"
5,投资支付的现金,,
6,质押贷款净增加额,,
7,取得子公司及其他营业单位支\n付的现金净额,,"7,946,963.55"
8,支付其他与投资活动有关的现\n金,"671,000,000.00","518,000,000.00"
9,投资活动现金流出小计,"674,957,030.27","544,318,910.51"


In [70]:
# 还是洗数据

#查找数据各在第几行：

dfsize3a = df3a.shape[0]
dfsize3c = df3c.shape[0]

# 找经营活动产生的现金流量净额
for i in range(0,dfsize3a):
    if df3a.iloc[i][0] == "经营活动产生的现金流量净额":
        loc3a = i
        break
opcash_this = float(df3a.iloc[loc3a][1].replace(',',''))
opcash_last = float(df3a.iloc[loc3a][2].replace(',',''))

# 找筹资活动产生的现金流量净额
for i in range(0,dfsize3c):
    if df3c.iloc[i][0] == "筹资活动产生的现金流量净额":
        loc3c = i
        break
fincash_this = float(df3c.iloc[loc3c][1].replace(',',''))
fincash_last = float(df3c.iloc[loc3c][2].replace(',',''))

# 找投资活动产生的现金流量净额

if g == True: # 投资和经营在一张表上
    
    for i in range(0,dfsize3a):
        if df3a.iloc[i][0] == "投资活动产生的现金流量净额":
            loc3b = i
            break
 
    invcash_this = float(df3a.iloc[loc3b][1].replace(',',''))
    invcash_last = float(df3a.iloc[loc3b][2].replace(',',''))
            
elif g == False: # 投资和筹资在一张表上
    for i in range(0,dfsize3c):
        if df3c.iloc[i][0] == "投资活动产生的现金流量净额":
            loc3b = i
            break
    invcash_this = float(df3c.iloc[loc3b][1].replace(',',''))
    invcash_last = float(df3c.iloc[loc3b][2].replace(',',''))
    



In [71]:

# 计算同比增减

opcash_var = round(100*(opcash_this - opcash_last)/opcash_last,2)
invcash_var = round(100*(invcash_this - invcash_last)/invcash_last,2)
fincash_var = round(100*(fincash_this - fincash_last)/fincash_last,2)



# 判断增减
updown_cash={}

if opcash_this < 0 and opcash_last < 0 and abs(opcash_this) < abs(opcash_last) :
    updown_cash[0] = '净流出同比减少'
elif opcash_this < 0 and opcash_last < 0 and abs(opcash_this) > abs(opcash_last):
    updown_cash[0] = '净流出同比增加'
elif opcash_this < 0 and opcash_last > 0:
    updown_cash[0] = '净流出同比增加'
elif opcash_this > 0 and opcash_last < 0:
    updown_cash[0] = '净流出同比减少'

    
if invcash_this < 0 and invcash_last < 0 and abs(invcash_this) < abs(invcash_last) :
    updown_cash[1] = '净流出同比减少'
elif invcash_this < 0 and invcash_last < 0 and abs(invcash_this) > abs(invcash_last):
    updown_cash[1] = '净流出同比增加'
elif invcash_this < 0 and invcash_last > 0:
    updown_cash[1] = '净流出同比增加'
elif invcash_this > 0 and invcash_last < 0:
    updown_cash[1] = '净流出同比减少'

    
if fincash_this < 0 and fincash_last < 0 and abs(fincash_this) < abs(fincash_last) :
    updown_cash[2] = '净流出同比减少'
elif fincash_this < 0 and fincash_last < 0 and abs(fincash_this) > abs(fincash_last):
    updown_cash[2] = '净流出同比增加'
elif fincash_this < 0 and fincash_last > 0:
    updown_cash[2] = '净流出同比增加'
elif fincash_this > 0 and fincash_last < 0:
    updown_cash[2] = '净流出同比减少'


liu_this = {}
liu_last = {}
n = 0
m = 0
for i in [opcash_this,invcash_this,fincash_this]:
    if i < 0:
        liu_this[n] = "流出"
    else:
        liu_this[n] = "流入"
    n = n + 1

for i in [opcash_last,invcash_last,fincash_last]:
    if i < 0:
        liu_last[m] = "流出"
    else:
        liu_last[m] = "流入"
    m = m + 1

In [72]:
cashline1 =  "经营活动现金净" +liu_this[0]+ mc(abs(opcash_this)) + ',' + updown_cash[0] + str(abs(opcash_var)) + '%' + "，去年同期净" + liu_last[0] + mc(abs(opcash_last)) + '；'
cashline2 =  "投资活动现金净" +liu_this[1]+ mc(abs(invcash_this)) + ',' + updown_cash[1] + str(abs(invcash_var)) + '%' + "，去年同期净" + liu_last[1] + mc(abs(invcash_last)) + '；'
cashline3 =  "筹资活动现金净" +liu_this[2]+ mc(abs(fincash_this)) + ',' + updown_cash[2] + str(abs(fincash_var)) + '%' + "，去年同期净" + liu_last[2] + mc(abs(fincash_last)) + '；'


In [73]:
# 财务摘要输出
print(line1_h1)
print(line2_h1)
print(line3_h1)
print(line4_h1)
print(line1_q2)
print(line2_q2)
print(line3_q2)
print(line4_q2)
print('\n')
#费用输出
print(rateline1)
print(rateline2)
print(rateline3)
print(rateline4)
print('\n')
#现金输出
print(cashline1)
print(cashline2)
print(cashline3)
print('\n')

20H1营业总收入6.02亿元,上年同期为6.57亿元,同比减少8.35%
20H1营业利润-419万元,上年同期为5687万元,同比减少-107.37%
20H1归母净利润-1474万元,上年同期为4022万元,同比减少136.66%
20H1扣非归母净利润-2329万元,上年同期为3189万元,同比减少173.04%
20Q2营业总收入3.71亿元,上年同期为4.02亿元,同比减少7.72%
20Q2营业利润4842万元,上年同期为6836万元,同比减少-29.16%
20Q2归母净利润3805万元,上年同期为5305万元,同比减少28.28%
20Q2扣非归母净利润3549万元,上年同期为4640万元,同比减少23.52%


销售费用3.28亿元，同比增加3.24%；销售费用率62.76%，较去年同期增加13.04pct；
管理费用9703万元，同比增加3.86%；管理费用率18.55%，较去年同期增加3.95pct；
研发费用7612万元，同比减少0.04%；研发费用率14.55%，较去年同期增加2.65pct；
财务费用-144万元，同比减少414.57%；财务费用率-0.28%，较去年同期减少0.35pct。


经营活动现金净流出8759万元,净流出同比增加194.58%，去年同期净流出2973万元；
投资活动现金净流入5517万元,净流出同比减少177.42%，去年同期净流出7126万元；
筹资活动现金净流出3148万元,净流出同比减少90.5%，去年同期净流出3.31亿元；




In [74]:
print(rateline1)
print(rateline2)
print(rateline3)
print(rateline4)
print('\n')

销售费用3.28亿元，同比增加3.24%；销售费用率62.76%，较去年同期增加13.04pct；
管理费用9703万元，同比增加3.86%；管理费用率18.55%，较去年同期增加3.95pct；
研发费用7612万元，同比减少0.04%；研发费用率14.55%，较去年同期增加2.65pct；
财务费用-144万元，同比减少414.57%；财务费用率-0.28%，较去年同期减少0.35pct。


